# LSTM with audio samples
THIS CODE IS NOT RUN. WE PROPOSE THE CODE SINCE WE HAVE TALKED ABOUT IT IN THE REPORT BUT THE RESULTS ARE VERY POOR AND THE COMPUTATIONAL COMPLEXITY HIGH SO IT WAS NOT RUN IN COLAB.

In this trial we decided to create a LSTM. The network take as input audio samples sampled with a frequency of 16 kHz.

In [ ]:
import numpy as np 
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook; tqdm.pandas()
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection import train_test_split
from scipy.io import wavfile
import csv
import random

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display
import keras
from keras import backend as K
from tensorflow.keras.layers import Layer, InputSpec
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import (Dense, LSTM,Dropout)
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)

from google.colab import drive
drive.mount('/content/drive')

---
Here we define the function to create the data windows that then we pass to the network. Due to the limited capacity of the hardware we had to downsample the files at 16 kHz

In [ ]:
def create_windows(signal, window_size, hop_length):
    num_samples = len(signal)
    num_windows = 1 + (num_samples - window_size) // hop_length
    windows = np.zeros((num_windows, window_size))

    for i in range(num_windows):
        start = i * hop_length
        end = start + window_size
        windows[i] = signal[start:end]

    return windows

def split_wav_in_windows(df):
    X = []
    window_size = 2048
    hop_length = 724
    for _,row in tqdm(df.iterrows()):
        y,sr = librosa.load(row.path, sr=16000)
        x = create_windows(y, window_size, hop_length)
        X.append(x.transpose())
    return X

---
Open the csv and create the train, validation and test sets.

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DB/splitted_train12.csv")
path = df["path"].values
labels = df["label"].values

# Preprocess dataset and create validation sets
X1 = np.array(convert_wav_to_image(df))

train_paths, test_paths, train_labels, test_labels = train_test_split(X1, labels, test_size=0.2, random_state=123)
train_paths, val_paths, train_labels, val_labels = train_test_split(train_paths, train_labels, test_size=0.15, random_state=123)

---
Here we create the network.

In [ ]:
input_shape=(2048,263)
model = keras.Sequential()
model.add(LSTM(96,input_shape=input_shape))
model.add(Dropout(0.3))
model.add(Dense(48, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

---
We define the early stopping mechanism, the batch size and the number of epochs. Then, we fit the model.

In [ ]:
learning_rate = 0.00001
optimizer = optimizers.Adam(learning_rate=learning_rate)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)
model.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [ ]:
batch_size=80
epochs = 140
history = model.fit(train_paths, train_labels, epochs=epochs, batch_size = batch_size,
                    validation_data=(val_paths, val_labels), shuffle=False, callbacks=[early_stopping])

---
Plotting the loss function and the accuracy for both train and validation set.

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_executed = early_stopping.stopped_epoch + 1
if epochs_executed != 1:
    rg = epochs_executed
else:
    rg = epochs    
plt.figure(figsize=(15,5))
plt.plot(range(rg), loss)
plt.plot(range(rg), val_loss)
plt.title('Loss over epochs', weight='bold', fontsize=22)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.legend(['Training loss', 'Validation loss'], fontsize=16)
plt.show()


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.figure(figsize=(15,5))
plt.plot(range(rg), acc)
plt.plot(range(rg), val_acc)
plt.title('Accuracy over epochs', weight='bold', fontsize=22)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.legend(['Training accuracy', 'Validation accuracy'], fontsize=16)
plt.show()

---
Evaluating the model on the test set, calculating the confusion matrix, the f1-score, the recall and the accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(test_paths, test_labels)

predictions = model.predict(test_paths)

# Convert the predicted probabilities to class labels (0 or 1 in this case)
y_pred = (predictions > 0.5).astype(int)

# Compute the confusion matrix
cm = confusion_matrix(test_labels, y_pred)

plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Reds)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Class 0', 'Class 1'])
plt.yticks(tick_marks, ['Class 0', 'Class 1'])
plt.xlabel('Predicted')
plt.ylabel('True')
# Add text annotations within each cell
thresh = cm.max() / 2
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.show()

#Printing the accuracy, the f1-score and the recall for each class
predicted_probabilities = predictions.flatten()
predicted_labels = (predicted_probabilities >= 0.5).astype(int)

report = classification_report(test_labels, predicted_labels)
print(report)